In [1]:
import os
import openai

In [2]:
openai.api_key = ""

In [3]:
def query(prompt, engine="davinci-codex", max_tokens=128, stop=None):
    if stop is not None:
        response = openai.Completion.create(
          engine=engine,
          prompt=prompt,
          temperature=0,
          max_tokens=max_tokens,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0,
          stop=stop
        )
    else:
        response = openai.Completion.create(
          engine=engine,
          prompt=prompt,
          temperature=0,
          max_tokens=max_tokens,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )
    return response

In [4]:
### Explaining an SQL query.
### In this example we use Codex to explain in a human readable format what an SQL query is doing.

prompt = '''SELECT DISTINCT department.name
FROM department
JOIN employee ON department.id = employee.department_id
JOIN salary_payments ON employee.id = salary_payments.employee_id
WHERE salary_payments.date BETWEEN '2020-06-01' AND '2020-06-30'
GROUP BY department.name
HAVING COUNT(employee.id) > 10;
-- Explanation of the above query in human readable format
--'''
response = query(prompt, max_tokens=256, stop=["\n\n"])
output = response.choices[0].text
print(output)

 Select the name of each department that has more than 10 employees who were paid in the month of June 2020
-- Join the tables department and employee on department.id = employee.department_id
-- Join the tables employee and salary_payments on employee.id = salary_payments.employee_id
-- Select the name of each department that has more than 10 employees who were paid in the month of June 2020
-- Group by the name of each department
-- Having the count of employees in each department is greater than 10


In [6]:
### Converting natural language to Postgres SQL

prompt = '''### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months
SELECT'''

response = query(prompt, max_tokens=150, stop=["#", ";"])
output = response.choices[0].text
print(output)

 DISTINCT department.name
FROM department
JOIN employee ON employee.department_id = department.id
JOIN salary_payments ON salary_payments.employee_id = employee.id
WHERE salary_payments.date > (CURRENT_DATE - INTERVAL '3 months')
GROUP BY department.name
HAVING COUNT(employee.id) > 10
